# Model Training: Churn Prediction (Using RFM + Segments)

## Amaç
Bu notebook’ta müşteri bazlı özellikleri (RFM ve opsiyonel segment) kullanarak
müşterinin churn (kaybedilmiş) olup olmadığını tahmin eden bir sınıflandırma modeli eğiteceğiz.

Not:
Churn etiketi veride doğrudan bulunmadığı için, Recency üzerinden iş kuralı ile üretilecektir.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Veri
Bir önceki adımda oluşturulan müşteri bazlı tablo (RFM + Cluster) kullanılacaktır.

In [2]:
rfm = pd.read_csv("../data/processed/customer_rfm.csv")
rfm.head()

,CustomerID,Recency,Frequency,Monetary,Recency_log,Frequency_log,Monetary_log
0,12346.0,326,1,77183.60,5.789960,0.693147,11.253955
1,12347.0,2,7,4310.00,1.098612,2.079442,8.368925
2,12348.0,75,4,1797.24,4.330733,1.609438,7.494564
3,12349.0,19,1,1757.55,2.995732,0.693147,7.472245
4,12350.0,310,1,334.40,5.739793,0.693147,5.815324


## Churn Tanımı (Label Engineering)
Churn etiketi veri setinde olmadığı için Recency üzerinden türetilir.
Recency değeri belirlenen eşiğin üzerindeyse müşteri churn=1 (kaybedilmiş) kabul edilir.

In [3]:
threshold = rfm["Recency"].quantile(0.80)  # 0.80 yerine 0.85 de deneyebiliriz
threshold

np.float64(179.0)

In [4]:
rfm["Churn"] = (rfm["Recency"] >= threshold).astype(int)
rfm["Churn"].value_counts(normalize=True)

Churn
0    0.799447
1    0.200553
Name: proportion, dtype: float64

## Churn Label Kontrolü ve Değerlendirme

### Churn Tanımı

Churn etiketi veri setinde doğrudan bulunmadığı için Recency değişkeni kullanılarak türetilmiştir.

Bu çalışmada:

- Recency değerinin %80 persentili eşik olarak belirlenmiştir.
- Recency değeri eşik değerinden büyük veya eşit olan müşteriler churn=1 (kaybedilmiş müşteri) olarak etiketlenmiştir.
- Eşik değer yaklaşık 179 gün olarak hesaplanmıştır.

Bu tanıma göre yaklaşık 6 ay boyunca alışveriş yapmayan müşteriler kaybedilmiş müşteri olarak kabul edilmiştir.

Bu yaklaşım e-commerce ve retail sektörlerinde yaygın olarak kullanılan müşteri kaybı tanımı ile uyumludur.

---

### Class Distribution (Sınıf Dağılımı)

Churn değişkeninin dağılımı incelendiğinde:

- Churn = 0 (aktif müşteriler): yaklaşık %80
- Churn = 1 (kaybedilmiş müşteriler): yaklaşık %20

Bu dağılım:

- Modelin öğrenmesi için yeterli veri sağlar
- Aşırı dengesiz veri problemi oluşturmaz
- Gerçek dünya müşteri davranışını temsil eder

---

### Sonuç

Oluşturulan churn etiketi iş kurallarına uygun, dengeli ve model eğitimi için uygun bulunmuştur.
Bu nedenle churn tahmin modeli oluşturma aşamasına geçilmiştir.

## Feature Selection (Model Giriş Değişkenleri)

Churn tahmini için müşteri davranışını temsil eden RFM özellikleri kullanılmıştır.

Kullanılan değişkenler:

- Recency_log → müşterinin son alışveriş zamanı
- Frequency_log → müşterinin alışveriş sıklığı
- Monetary_log → müşterinin toplam harcama miktarı

Bu değişkenler müşteri aktivitesini doğrudan temsil ettiği için churn tahmini için uygun özelliklerdir.

In [12]:
features = ["Frequency_log","Monetary_log"]

X = rfm[features]
y = rfm["Churn"]

X.head()

,Frequency_log,Monetary_log
0,0.693147,11.253955
1,2.079442,8.368925
2,1.609438,7.494564
3,0.693147,7.472245
4,0.693147,5.815324


## Train-Test Split

Model performansını objektif olarak değerlendirmek için veri eğitim ve test
setlerine ayrılmıştır.

- Eğitim verisi → model öğrenir
- Test verisi → model performansı ölçülür

Veri %80 eğitim, %20 test olacak şekilde ayrılmıştır.

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)

(3470, 2) (868, 2)


## Feature Scaling

Model performansını artırmak için değişkenler StandardScaler kullanılarak
ölçeklenmiştir. Bu işlem değişkenlerin ortalamasını 0 ve standart sapmasını 1 yapar.

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Logistic Regression Model

Churn tahmini için başlangıç modeli olarak Logistic Regression kullanılmıştır.
Bu model hızlı, yorumlanabilir ve sınıflandırma problemleri için yaygın kullanılan bir yöntemdir.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)

y_pred = lr.predict(X_test_scaled)
y_proba = lr.predict_proba(X_test_scaled)[:,1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.80      0.99      0.89       694
           1       0.60      0.03      0.07       174

    accuracy                           0.80       868
   macro avg       0.70      0.51      0.48       868
weighted avg       0.76      0.80      0.72       868

ROC AUC: 0.7926521249461724


## Random Forest Model

Random Forest algoritması non-linear ilişkileri öğrenebilen güçlü bir ensemble yöntemidir.
Churn tahmini performansını artırmak için kullanılmıştır.

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       694
           1       0.35      0.34      0.34       174

    accuracy                           0.74       868
   macro avg       0.59      0.59      0.59       868
weighted avg       0.74      0.74      0.74       868

ROC AUC: 0.701840902315413


## Data Leakage Önleme

Churn etiketi Recency değişkeni kullanılarak oluşturulduğu için,
Recency değişkeni model giriş özelliklerinden çıkarılmıştır.

Aksi durumda model hedef değişkeni doğrudan öğrenebilir ve
gerçekçi olmayan performans sonuçları elde edilir.